In [8]:
from sympy import  factorint
import math,string,random
import numpy as np

In [12]:
# u must be great than v
def EuclideEtendu(u,v):
    liste1 = [u,1,0]
    liste2= [v , 0,1]    
    while liste2[0]:
        q,r = divmod(liste1[0],liste2[0])
        tmp_list = liste2.copy()
        liste2[0],liste2[1],liste2[2] = r ,liste1[1] -q*tmp_list[1] ,liste1[2] -q*tmp_list[2]
        liste1 = tmp_list.copy()
    return liste1[0],liste1[1],liste1[2]

def inverse(max, min):
    _,_, v = EuclideEtendu(max, min) 
    if v < 0:
        return v + max
    return v

     

In [13]:
# liste = [(a,n),(b,m)] avec pgcd(m,n) = 1
def restChinoi_1(liste):
    n ,m= liste[0][1] ,liste[1][1]
    mn = m * n
    _,nombre_1 , nombre_2 = EuclideEtendu(m,n)
    return (m*nombre_1*liste[0][0] + n * nombre_2*liste[1][0]) % mn



# liste = [(a_i,m_i) ,( , ), . . .] avec pgcd(m_i,j_j) = 1 pour tout j
def restChinoi_2(liste):
    while len( liste ) != 2:
        M= liste [0][1]*liste[1][1]
        tmp_liste = [liste[0] , liste[1]]
        c = restChinoi_1(tmp_liste) % M
        liste = liste[2:]
        liste= [(c,M)] + liste 
    tmp_list = [liste[0] , liste[1]]
    return restChinoi_1(tmp_list)

In [14]:
def pohlig_hellman1(g, h, p, n):
    # Factorisation de n
    factors = factorint(n) 
    
    # Résolution modulaire pour chaque facteur premier
    x_list = []
    moduli = []
    
    for q, e in factors.items():
        q_e = q ** e
        g_q = pow(g, n // q_e, p)  # g^(n / q^e) mod p
        h_q = pow(h, n // q_e, p)  # h^(n / q^e) mod p
        
        # Résolution dans le sous-groupe d'ordre q^e
        x_q = solve_baby_step_giant_step(g_q, h_q, q_e, p)
        x_list.append(x_q)
        moduli.append(q_e)
    
    # Reconstruction de x avec le théorème des restes chinois
    liste= [(u,y) for u,y in zip(x_list,moduli)]
    x = restChinoi_2(liste)
    
    return x
# Algorithme Baby-step Giant-step pour résoudre g^x ≡ h mod p
# param n: Ordre du groupe
def solve_baby_step_giant_step(g, h, n, p):
    
    m = int(math.ceil(math.sqrt(n)))
    table= {} # Calcul des baby steps
    for j in range(m):
        value = pow(g, j, p)
        table[value] = j

    # Calcul des giant steps
    g_inv_m = pow(g, -m, p) 
    gamma = h
    for i in range(m):
        if gamma in table:
            return i * m + table[gamma]
        gamma = (gamma * g_inv_m) % p

    raise ValueError("Logarithme discret non trouvé.")



In [15]:
def decodage(message_):
    cle_de_decodage={str(x + 1):string.printable[x] for x in range(10,36)}
    cle_de_decodage['42']="'"
    cle_de_decodage['43']="."
    cle_de_decodage['44']=","
    cle_de_decodage['45']="?"
    message_= str(message_)
    result=''
    for i in range(len(message_)//2):
        if not message_[2*i:2*i+2] in cle_de_decodage.keys():
            result = result + ' '
        else:
            result = result + cle_de_decodage[message_[2*i:2*i+2]]
    return result

In [42]:
def Decriptage_El_Gamal(message_cripte,p,a):
    u,v =message_cripte[0],message_cripte[1]
    return (v*pow(inverse(p,u) ,a))% p

## Exercice 2 : 

In [39]:
p=112211293740262525327
gA=2014
gB=2022
pohlig_hellman1(3,gA,p,p-1)


39337370770196487501

## Exercice 3: Decodage de message d'Alice

In [45]:

message=[[83025882561049910713,66740266984208729661],[117087132399404660932,44242256035307267278],[67508282043396028407,77559274822593376192],[60938739831689454113,14528504156719159785],[5059840044561914427,59498668430421643612],[92232942954165956522,105988641027327945219],[97102226574752360229,46166643538418294423]]
p=123456789987654353003
g=123456789
gA=52808579942366933355
gB=39318628345168608817
key = (p,g,gB,p)

message = [(message[i][0],message[i][1]) for i in range(len(message))]

In [44]:
a=pohlig_hellman1(g,gB,p,p-1)

### Message :

In [48]:

message_=''
for mes in message:
    message_= message_ + decodage(Decriptage_El_Gamal(mes,p,a))

print(message_)

in galois fields, full of flowers, primitive elements dance for hours.


# Exercice 4:

In [2]:
def Miller_Rabin(n,temoin):
    m=n
    if n % 2 == 0:
        return False
    n=n-1
    inc = 0
    while n % 2 ==0:
        n= n // 2
        inc +=1
    reste=int(n)
    result= pow(temoin,reste,m)
    if result == 1 or result == m -1:
        return True
    for _ in range(inc):
        result=pow(result,2,m)
        if result == m-1:
            return True
    return False


def Miller_Rabin_repeated(nombre,repetition):
    for _ in range(repetition):
        temoin=random.randint(2,nombre-1)
        if not Miller_Rabin(nombre,temoin):
            return False
    return True

In [25]:
def Safe_prime(n):
    x=pow(2,n)
    # y0=(x-int(pow((math.log(x)),2)))/2*int(pow(math.log(x),2))
    y0 = (x - int(pow(math.log(x),2)))//(2*(int(pow(math.log(x),2))))
    y1=int(pow(2,n-1) -1)
    while True:
        l=random.randint(y0,y1)
        if  Miller_Rabin_repeated(l,5) and Miller_Rabin_repeated(2*l + 1,5):
            return 2*l + 1
        for j in range(int(n/4),int(n/2)):
            l1=random.randint(pow(2,j) +1,pow(2,j+1))
            l2=random.randint(int(y0/l1),int(y1/l1) +1)
            if Miller_Rabin_repeated(l1,5) and Miller_Rabin_repeated(l2,5) and Miller_Rabin_repeated(2*l1*l2 +1,5) :
                return 2*l1*l2 + 1

## Safe prime:

In [26]:
prime = Safe_prime(1330)

In [27]:
prime

6663375793016645420425836919425834275219728969140910373873486641041912664035407129496687306021138724560332119898948805231749704212061513770567728893853360680943718189622298771800575754674623313035572350286590432033965769301458205111083065217667648260153367524892401577416802565708642234283443398334305538027584606060394109260290444609902213919254912137985427974607573755142346405129333108238107405819

In [34]:
def find_generator(p): # p est un nombre premier
    n=p-1
    inc = 0
    while n % 2 ==0:
        n= n // 2
        inc +=1
    reste=int(n)
    generator = random.randint(2,p-1)
    while pow(generator,(p-1)//2,p)==1 and pow(generator,(p-1) // reste, p) == 1:
        generator = random.randint(2,p-1)
    return generator

In [37]:
generator=find_generator(prime)
generator

37094259169674586858018907083307942132401152939191201659282016329640609892521764312029846447969374860491183636605163843104705645485634127401000162866691449026734526615470215115187569085778736199006488302566889261856310089214527370633087167178003952424162044691593180157815992138969576812138508244193629758187579007278654035911233303176013685269394782986428068712036790963214998480509261548872388214

In [31]:
def find_keys(generator,p):
    cle_prive = random.randint(2,p-2)
    cle_publique = pow(generator,cle_prive,p)

    return cle_prive ,cle_publique
    

# Cle privé:
$a = 4191900185582565814551051166790932236137481238407438825905515618064726614909149595960339352433303679462574213612937174254562176638851706565375337481392895213843621698445931735479363685816345999117858467775882165356442229550927011950073952878897172797622905504833919869355333345765883820056005841927989378669743042286551893341013996519197540040403815898897832871042031407936151984378563390576318022141$


In [40]:
find_keys(generator,prime)[0]

4191900185582565814551051166790932236137481238407438825905515618064726614909149595960339352433303679462574213612937174254562176638851706565375337481392895213843621698445931735479363685816345999117858467775882165356442229550927011950073952878897172797622905504833919869355333345765883820056005841927989378669743042286551893341013996519197540040403815898897832871042031407936151984378563390576318022141

# Cle publique  :
- $G = \mathbb{(Z/qZ)}$  avec $q=666337579301664542042583691942583427521972896914091037387348664104191266403540712949668730602113872456033211989894880523174970421206151377056772889385336
0680943718189622298771800575754674623313035572350286590432033965769301458205111083065217667648260153367524892401577416802565708642234283443398334305538027584606060394109260290444609902213919254912137985427974607573755142346405129333108238107405819$

- $g = 37094259169674586858018907083307942132401152939191201659282016329640609892521764312029846447969374860491183636605163843104705645485634127401000162866691449026734526615470215115187569085778736199006488302566889261856310089214527370633087167178003952424162044691593180157815992138969576812138508244193629758187579007278654035911233303176013685269394782986428068712036790963214998480509261548872388214$

- $g^{a} =1632884228264797632916577868616196352915683373377000478262320888184668115666712088058353672761653089446548139746061510280646142444578470409634096094501741387315300179542537613580783630762088718142152488570370890474818446728591551750204002032975412317437738514839591138996263395105205316794115952950778057328493689233611558640806340317679221358686004251327486417054900505800123218074007350006083966952$

In [39]:
find_keys(generator,prime)[1]

1632884228264797632916577868616196352915683373377000478262320888184668115666712088058353672761653089446548139746061510280646142444578470409634096094501741387315300179542537613580783630762088718142152488570370890474818446728591551750204002032975412317437738514839591138996263395105205316794115952950778057328493689233611558640806340317679221358686004251327486417054900505800123218074007350006083966952